
# Advanced Recommender Systems with Python

Welcome to the code notebook for creating Advanced Recommender Systems with Python. This is an optional lecture notebook for you to check out. Currently there is no video for this lecture because of the level of mathematics used and the heavy use of SciPy here.

Recommendation Systems usually rely on larger data sets and specifically need to be organized in a particular fashion. Because of this, we won't have a project to go along with this topic, instead we will have a more intensive walkthrough process on creating a recommendation system with Python with the same Movie Lens Data Set.

*Note: The actual mathematics behind recommender systems is pretty heavy in Linear Algebra.*
___

## Methods Used

Two most common types of recommender systems are **Content-Based** and **Collaborative Filtering (CF)**. 

* Collaborative filtering produces recommendations based on the knowledge of users’ attitude to items, that is it uses the "wisdom of the crowd" to recommend items. 
* Content-based recommender systems focus on the attributes of the items and give you recommendations based on the similarity between them.

## Collaborative Filtering

In general, Collaborative filtering (CF) is more commonly used than content-based systems because it usually gives better results and is relatively easy to understand (from an overall implementation perspective). The algorithm has the ability to do feature learning on its own, which means that it can start to learn for itself what features to use. 

CF can be divided into **Memory-Based Collaborative Filtering** and **Model-Based Collaborative filtering**. 

In this tutorial, we will implement Model-Based CF by using singular value decomposition (SVD) and Memory-Based CF by computing cosine similarity. 

## The Data

We will use famous MovieLens dataset, which is one of the most common datasets used when implementing and testing recommender engines. It contains 100k movie ratings from 943 users and a selection of 1682 movies.

You can download the dataset [here](http://files.grouplens.org/datasets/movielens/ml-100k.zip) or just use the u.data file that is already included in this folder.

____
## Getting Started

Let's import some libraries we will need:

In [1]:
import numpy as np
import pandas as pd

We can then read in the **u.data** file, which contains the full dataset. You can read a brief description of the dataset [here](http://files.grouplens.org/datasets/movielens/ml-100k-README.txt).

Note how we specify the separator argument for a Tab separated file.

In [2]:
column_names = ['user_id', 'item_id', 'rating', 'timestamp']
df = pd.read_csv('u.data', sep='\t', names=column_names)

Let's take a quick look at the data.

In [3]:
df.head()

,user_id,item_id,rating,timestamp
0,0,50,5,881250949
1,0,172,5,881250949
2,0,133,1,881250949
3,196,242,3,881250949
4,186,302,3,891717742


Note how we only have the item_id, not the movie name. We can use the Movie_ID_Titles csv file to grab the movie names and merge it with this dataframe:

In [4]:
movie_titles = pd.read_csv("Movie_Id_Titles")
movie_titles.head()

,item_id,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


Then merge the dataframes:

In [5]:
df = pd.merge(df,movie_titles,on='item_id')
df.head()

,user_id,item_id,rating,timestamp,title
0,0,50,5,881250949,Star Wars (1977)
1,290,50,5,880473582,Star Wars (1977)
2,79,50,4,891271545,Star Wars (1977)
3,2,50,5,888552084,Star Wars (1977)
4,8,50,5,879362124,Star Wars (1977)


Now let's take a quick look at the number of unique users and movies.

In [6]:
n_users = df.user_id.nunique()
n_items = df.item_id.nunique()

print('Num. of Users: '+ str(n_users))
print('Num of Movies: '+str(n_items))

Num. of Users: 944
Num of Movies: 1682


## Train Test Split

Recommendation Systems by their very nature are very difficult to evaluate, but we will still show you how to evaluate them in this tutorial. In order to do this, we'll split our data into two sets. However, we won't do our classic X_train,X_test,y_train,y_test split. Instead we can actually just segement the data into two sets of data:

In [7]:
from sklearn.cross_validation import train_test_split
train_data, test_data = train_test_split(df, test_size=0.25)

/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## Memory-Based Collaborative Filtering

Memory-Based Collaborative Filtering approaches can be divided into two main sections: **user-item filtering** and **item-item filtering**. 

A *user-item filtering* will take a particular user, find users that are similar to that user based on similarity of ratings, and recommend items that those similar users liked. 

In contrast, *item-item filtering* will take an item, find users who liked that item, and find other items that those users or similar users also liked. It takes items and outputs other items as recommendations. 

* *Item-Item Collaborative Filtering*: “Users who liked this item also liked …”
* *User-Item Collaborative Filtering*: “Users who are similar to you also liked …”

In both cases, you create a user-item matrix which built from the entire dataset.

Since we have split the data into testing and training we will need to create two ``[943 x 1682]`` matrices (all users by all movies). 

The training matrix contains 75% of the ratings and the testing matrix contains 25% of the ratings.  

Example of user-item matrix:
<img class="aligncenter size-thumbnail img-responsive" src="http://s33.postimg.org/ay0ty90fj/BLOG_CCA_8.png" alt="blog8"/>

After you have built the user-item matrix you calculate the similarity and create a similarity matrix. 

The similarity values between items in *Item-Item Collaborative Filtering* are measured by observing all the users who have rated both items.  

<img class="aligncenter size-thumbnail img-responsive" style="max-width:100%; width: 50%; max-width: none" src="http://s33.postimg.org/i522ma83z/BLOG_CCA_10.png"/>

For *User-Item Collaborative Filtering* the similarity values between users are measured by observing all the items that are rated by both users.

<img class="aligncenter size-thumbnail img-responsive" style="max-width:100%; width: 50%; max-width: none" src="http://s33.postimg.org/mlh3z3z4f/BLOG_CCA_11.png"/>

A distance metric commonly used in recommender systems is *cosine similarity*, where the ratings are seen as vectors in ``n``-dimensional space and the similarity is calculated based on the angle between these vectors. 
Cosine similiarity for users *a* and *m* can be calculated using the formula below, where you take dot product of  the user vector *$u_k$* and the user vector *$u_a$* and divide it by multiplication of the Euclidean lengths of the vectors.
<img class="aligncenter size-thumbnail img-responsive" src="https://latex.codecogs.com/gif.latex?s_u^{cos}(u_k,u_a)=\frac{u_k&space;\cdot&space;u_a&space;}{&space;\left&space;\|&space;u_k&space;\right&space;\|&space;\left&space;\|&space;u_a&space;\right&space;\|&space;}&space;=\frac{\sum&space;x_{k,m}x_{a,m}}{\sqrt{\sum&space;x_{k,m}^2\sum&space;x_{a,m}^2}}"/>

To calculate similarity between items *m* and *b* you use the formula:

<img class="aligncenter size-thumbnail img-responsive" src="https://latex.codecogs.com/gif.latex?s_u^{cos}(i_m,i_b)=\frac{i_m&space;\cdot&space;i_b&space;}{&space;\left&space;\|&space;i_m&space;\right&space;\|&space;\left&space;\|&space;i_b&space;\right&space;\|&space;}&space;=\frac{\sum&space;x_{a,m}x_{a,b}}{\sqrt{\sum&space;x_{a,m}^2\sum&space;x_{a,b}^2}}
"/>

Your first step will be to create the user-item matrix. Since you have both testing and training data you need to create two matrices.  

In [32]:
#Create two user-item matrices, one for training and another for testing
train_data_matrix = np.zeros((n_users, n_items))
for line in train_data.itertuples():
    train_data_matrix[line[1]-1, line[2]-1] = line[3]  

test_data_matrix = np.zeros((n_users, n_items))
for line in test_data.itertuples():
    test_data_matrix[line[1]-1, line[2]-1] = line[3]

In [35]:
train_data_matrix = np.zeros((n_users, n_items))
for line in train_data.itertuples():
    train_data_matrix[line[1]-1, line[2]-1] = line[3]
    print(line[0],line[1],line[2],line[3],line[4])

58848 280 13 5 891700257
45206 727 250 5 883709242
75254 514 429 4 875311225
95757 11 713 5 891903024
54098 79 306 5 891271792
31626 62 554 1 879375562
90400 314 993 5 877886279
40146 328 751 3 885596088
84165 694 630 3 875728912
63822 293 696 2 888905229
68507 586 397 3 884063080
79263 131 276 5 883681723
79567 479 647 5 879461039
71319 653 380 3 880151984
82232 715 470 4 875963538
4554 20 1 3 879667963
11234 812 288 4 877625294
13182 773 229 3 888540112
74504 102 435 3 888801315
30094 655 178 4 887427009
14546 70 625 3 884151316
80236 270 581 5 876955938
91958 901 391 5 877131205
35511 374 595 3 880393921
24214 435 559 3 884132342
67637 56 91 4 892683275
97948 279 1087 2 891209189
9849 694 196 5 875727226
61186 72 509 4 880036638
68643 561 513 3 885807345
76592 258 300 5 885700877
44189 42 12 4 881107502
23503 261 117 4 890455974
13879 654 54 3 887864941
87429 21 628 3 874951616
69191 846 87 4 883948417
52968 642 168 5 885842943
69967 450 73 3 887661438
57642 236 1039 2 890115996
317

48548 415 531 5 879439684
24830 72 2 3 880037376
83294 637 1284 1 882905070
60692 716 492 3 879795425
74484 690 435 5 881177616
98625 733 1338 4 879536608
55985 494 121 4 879541429
2036 513 265 5 885062919
26570 770 258 5 875971568
37344 799 484 3 879254077
89864 110 947 3 886988574
70899 39 319 4 891400094
47364 499 173 5 885599307
51388 89 187 5 879461246
12593 897 416 5 879991186
85886 267 810 3 878973568
71479 416 585 1 886318085
60832 524 161 4 884637095
76824 324 411 5 880575589
43401 285 286 3 890595584
42332 936 333 3 886831415
3897 337 392 5 875236512
75419 567 1019 5 882425874
20327 593 385 4 886194041
76523 940 300 5 884801316
32352 621 568 5 874963797
51785 836 289 1 885753691
94416 89 694 5 879460027
79523 59 647 5 888205336
32999 201 68 2 884112487
44575 49 462 2 888066486
37744 59 215 5 888204430
93966 561 362 2 893105375
69224 334 72 3 891549192
95255 537 646 2 886030552
61425 14 820 3 882839856
54528 886 943 3 876032248
20214 186 385 4 879023894
97623 624 881 3 8797921

49489 387 211 4 886480108
98209 585 1524 3 891283124
82265 896 470 2 887159531
59680 290 692 5 880474293
58252 551 809 5 892784629
20119 593 200 5 875661567
60650 916 1220 3 880845282
59243 290 739 3 880475757
48789 343 1047 1 876403776
32619 460 591 2 882911603
60298 417 49 3 880951737
48408 752 325 2 891208126
21341 447 471 4 878854340
8570 577 100 4 880470350
85722 268 1059 3 875743310
20763 486 245 3 879875441
53979 711 381 5 879994749
17679 551 132 5 892777583
21313 921 471 2 879379821
69121 643 87 5 891447663
16818 707 208 5 886285939
16692 548 294 3 891042954
51679 704 289 3 891396881
54899 151 58 4 879524849
98588 733 534 3 879544377
89119 452 456 1 876209837
53757 152 720 5 882477356
10419 122 423 4 879270805
68672 406 513 5 879445378
30814 59 106 4 888203959
28639 832 748 3 888259984
74092 655 953 3 887427243
14255 756 403 2 874828826
70429 412 634 5 879716918
72617 676 682 1 892685716
24028 308 273 2 887737084
44821 457 845 4 882393801
65251 682 9 3 888517168
45620 234 71 3 

8239 99 332 3 885678348
78486 886 1048 4 876032840
61790 846 780 4 883949380
37336 481 484 4 885828686
43964 567 640 4 882426927
20239 401 385 3 891033603
58862 634 13 4 878916178
2616 38 257 1 892429512
90352 157 1132 3 886891132
43828 562 402 5 879196074
46169 881 176 4 876537679
1673 624 346 3 885215462
1389 474 302 5 887914615
25348 717 281 4 884642958
75163 314 1041 4 877888445
12153 943 427 4 888639147
90617 292 653 4 881105442
84566 532 1199 3 874789155
23052 654 405 4 887863866
84405 313 615 4 891013652
63617 917 287 4 882911185
66073 566 122 2 881651583
66131 119 310 5 886175117
26967 115 673 3 881171558
69900 297 73 2 875239691
61850 543 44 3 874865728
75692 328 693 2 885046174
87660 815 31 4 878695490
6467 676 193 5 892686606
40292 894 256 3 879896704
56520 130 284 2 874953728
56956 18 22 5 880130640
77153 363 657 5 891494587
2329 301 451 4 882078061
27359 655 216 4 887428086
80730 851 313 4 883148627
62262 716 729 2 879795375
42914 158 175 4 880135044
98034 181 985 1 878962

19970 537 195 3 886031407
3545 741 785 3 891457371
41629 454 199 3 881960413
1193 205 242 4 888284313
13472 694 237 4 875728509
31593 634 1028 3 875729456
71019 333 180 1 891045191
84935 344 311 4 884814359
943 533 172 4 879191184
51481 663 187 5 889493869
47659 416 17 2 886318084
64664 445 324 1 891199297
59384 280 739 3 891701359
12935 825 248 4 880755869
9684 846 181 5 883947694
13251 705 229 3 883428154
16309 133 294 3 890588852
76453 857 300 3 883432251
90888 621 1118 3 874962824
95458 92 934 2 875639642
19199 699 21 3 884152916
51120 332 182 5 888098088
71232 178 1033 2 882824869
28863 606 255 5 887061723
4392 403 118 5 879785974
47472 256 460 4 882153987
82067 497 808 2 879310941
33774 551 334 4 892775970
23387 854 268 3 882811865
43913 922 402 3 891448451
33838 548 344 1 891042530
1945 932 474 5 891250418
60922 932 161 3 891251507
63065 239 433 5 889180447
7216 655 796 2 887428280
73812 251 24 3 886272283
39816 71 134 3 885016614
15138 286 154 4 877533381
46495 450 716 4 882469

...

10396 295 423 4 879517372
81491 875 772 5 876465188
90674 429 45 3 882385118
13761 716 480 5 879795025
80648 26 313 5 891386368
7572 214 327 5 892668196
81071 387 205 5 886480384
37591 940 508 5 885921198
54674 943 824 4 875502483
31964 214 318 4 892668249
45910 878 781 1 880871600
35910 326 82 3 879876861
83241 87 575 3 879877208
47371 933 173 3 874855020
24108 244 559 4 880607154
63130 378 99 4 880045791
69554 336 125 3 877760032
74655 363 223 5 891495197
72069 210 792 3 887730532
80476 72 553 5 880036638
77514 863 898 1 889288973
49802 750 305 4 879445877
88690 342 584 4 874984430
62032 445 28 4 890987772
92247 417 638 4 879648078
7801 447 201 2 878855723
40065 250 751 2 883262694
3674 403 274 3 879786661
99191 49 1080 4 888066734
71727 650 737 2 891383832
98121 279 1219 3 875744358
99581 655 1553 4 888474019
20474 526 750 4 885681886
65109 747 129 5 888639138
86026 21 769 1 874951916
1814 488 474 2 891294439
69989 647 73 5 876537697
65198 354 9 3 891216607
74328 621 779 3 880740296

56642 330 284 5 876544311
99698 844 1474 4 877387195
37151 796 357 4 892662400
4818 388 1 5 886436813
47737 646 315 4 888528483
78668 280 499 4 891700496
65418 10 9 4 877889005
59439 747 661 5 888639642
57438 541 931 3 883875370
11619 479 526 4 879461378
10037 256 679 3 882164525
26437 100 258 4 891374675
29179 579 202 5 880952270
72592 381 682 2 892697982
15905 342 209 5 875319554
21060 918 135 1 891986634
44223 305 12 5 886322930
59409 60 661 4 883326808
82483 319 340 3 879975481
90588 600 562 3 888452564
41734 90 709 5 891383752
62170 128 28 5 879966785
57187 894 961 4 882404642
17591 698 496 3 886366690
64954 72 461 3 880036824
79740 414 886 4 884999286
13439 95 237 2 879192708
76012 733 253 3 879535407
16720 233 208 4 880610814
93473 181 1351 1 878962168
78989 734 83 4 891022733
82136 6 535 2 883600030
2537 855 86 2 879825462
37328 385 484 4 879442559
5829 436 234 3 887769471
42614 172 425 1 875536591
67234 193 155 4 889126376
41261 878 197 4 880866971
76250 294 300 4 877818861
54

12819 913 690 3 880824288
17225 529 328 4 882535256
96112 585 639 4 891283921
74138 253 510 5 891628416
94506 477 1051 5 875941763
2149 746 265 4 885075399
7579 46 327 4 883611456
47832 334 505 4 891546405
25498 548 298 4 891043882
14016 655 518 2 888813186
13192 498 229 2 881961877
89467 916 636 3 880845391
3755 374 274 4 880393668
94725 270 563 3 876956442
61431 601 820 1 876348316
34487 22 89 5 878887680
52955 178 168 4 882826347
47788 894 315 4 885428012
53447 637 150 1 882903447
8204 116 332 3 876451998
46729 201 699 3 884140610
52739 886 732 3 876032029
47285 545 173 5 879900185
81317 592 192 5 882955460
57935 56 70 4 892676996
50060 870 210 4 879270313
21147 207 1147 4 879665042
74243 299 510 5 889501392
59022 345 588 3 884992100
41637 325 199 5 891478199
5592 151 234 4 879524819
95976 871 896 3 888192858
77967 6 308 3 883600445
27701 321 48 4 879439706
711 456 172 5 881373019
99540 243 1465 3 879988215
93358 708 358 2 892719007
52285 66 742 5 883601388
36255 293 501 4 888906378

48312 220 325 1 881198435
3434 5 29 4 875637023
44515 455 291 3 879109984
18895 241 307 4 887249795
9434 371 181 3 877486953
24020 595 273 3 886921140
29522 447 235 2 878854605
98609 887 1279 3 881378402
64841 37 92 4 880930072
65591 230 233 1 880485513
42425 654 1115 3 887863779
16287 248 294 3 884534379
29773 738 228 5 875350316
44795 193 845 4 889124803
10910 144 20 4 888104559
98505 13 914 2 892870589
23529 95 117 4 879193619
23393 747 268 5 888638091
18167 527 174 4 879455847
80339 30 539 3 885941454
55057 896 58 3 887159531
88996 880 689 4 880166577
42436 286 577 2 877535500
87080 499 326 3 892501059
17112 317 328 4 891446438
88292 293 420 4 888907356
56800 296 22 4 884197068
32939 442 746 3 883388354
5235 244 95 4 880606418
32531 621 65 3 885596944
94570 558 19 5 879436396
13891 650 54 2 891385876
51965 796 185 4 893194548
38282 698 204 2 886366770
37718 818 303 5 891870222
99642 181 1381 2 878962349
52921 298 168 5 884182933
78967 450 83 4 882372027
45713 536 71 5 882360467
369

86439 267 464 5 878974783
69978 476 73 4 883364475
25420 532 1240 2 874793852
438 232 50 4 880062302
73597 419 488 5 879435722
19414 889 514 1 880178158
25312 378 281 3 880044609
26000 399 79 3 882512214
12283 422 15 3 875129882
96991 506 1244 2 884517295
55387 872 905 4 888479034
15440 763 275 5 878915958
93584 709 633 3 879846561
73777 276 24 4 874792366
99625 782 1511 2 891500194
7176 908 603 4 879722361
34690 374 576 3 880939186
93595 848 633 3 887043040
57270 115 240 5 881171982
71262 94 380 3 891722760
68133 387 93 5 886480703
6587 505 88 4 889334334
15350 18 275 5 880129421
83577 534 760 2 877808098
99385 303 1426 2 879484804
62817 889 566 3 880181275
50523 117 156 4 881011376
97455 295 1446 4 879519026
45559 618 186 4 891307224
31348 44 214 5 878348036
50270 551 770 2 892778244
91280 413 270 4 879969027
537 464 50 4 878354966
74411 426 435 3 879444604
81553 279 42 4 875308843
2992 1 222 4 878873388
68785 741 38 2 891455832
12577 843 416 2 879448352
75329 13 868 5 882139901
8441

You can use the [pairwise_distances](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.pairwise_distances.html) function from sklearn to calculate the cosine similarity. Note, the output will range from 0 to 1 since the ratings are all positive.

In [9]:
from sklearn.metrics.pairwise import pairwise_distances
user_similarity = pairwise_distances(train_data_matrix, metric='cosine')
item_similarity = pairwise_distances(train_data_matrix.T, metric='cosine')

Next step is to make predictions. You have already created similarity matrices: `user_similarity` and `item_similarity` and therefore you can make a prediction by applying following formula for user-based CF:

<img class="aligncenter size-thumbnail img-responsive" src="https://latex.codecogs.com/gif.latex?\hat{x}_{k,m}&space;=&space;\bar{x}_{k}&space;&plus;&space;\frac{\sum\limits_{u_a}&space;sim_u(u_k,&space;u_a)&space;(x_{a,m}&space;-&space;\bar{x_{u_a}})}{\sum\limits_{u_a}|sim_u(u_k,&space;u_a)|}"/>

You can look at the similarity between users *k* and *a* as weights that are multiplied by the ratings of a similar user *a* (corrected for the average rating of that user). You will need to normalize it so that the ratings stay between 1 and 5 and, as a final step, sum the average ratings for the user that you are trying to predict. 

The idea here is that some users may tend always to give high or low ratings to all movies. The relative difference in the ratings that these users give is more important than the absolute values. To give an example: suppose, user *k* gives 4 stars to his favourite movies and 3 stars to all other good movies. Suppose now that another user *t* rates movies that he/she likes with 5 stars, and the movies he/she fell asleep over with 3 stars. These two users could have a very similar taste but treat the rating system differently. 

When making a prediction for item-based CF you don't need to correct for users average rating since query user itself is used to do predictions.

<img class="aligncenter size-thumbnail img-responsive" src="https://latex.codecogs.com/gif.latex?\hat{x}_{k,m}&space;=&space;\frac{\sum\limits_{i_b}&space;sim_i(i_m,&space;i_b)&space;(x_{k,b})&space;}{\sum\limits_{i_b}|sim_i(i_m,&space;i_b)|}"/>

In [10]:
def predict(ratings, similarity, type='user'):
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1)
        #You use np.newaxis so that mean_user_rating has same format as ratings
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis]) 
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])     
    return pred

In [11]:
item_prediction = predict(train_data_matrix, item_similarity, type='item')
user_prediction = predict(train_data_matrix, user_similarity, type='user')

### Evaluation
There are many evaluation metrics but one of the most popular metric used to evaluate accuracy of predicted ratings is *Root Mean Squared Error (RMSE)*. 
<img src="https://latex.codecogs.com/gif.latex?RMSE&space;=\sqrt{\frac{1}{N}&space;\sum&space;(x_i&space;-\hat{x_i})^2}" title="RMSE =\sqrt{\frac{1}{N} \sum (x_i -\hat{x_i})^2}" />

You can use the [mean_square_error](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html) (MSE) function from `sklearn`, where the RMSE is just the square root of MSE. To read more about different evaluation metrics you can take a look at [this article](http://research.microsoft.com/pubs/115396/EvaluationMetrics.TR.pdf). 

Since you only want to consider predicted ratings that are in the test dataset, you filter out all other elements in the prediction matrix with `prediction[ground_truth.nonzero()]`. 

In [12]:
from sklearn.metrics import mean_squared_error
from math import sqrt
def rmse(prediction, ground_truth):
    prediction = prediction[ground_truth.nonzero()].flatten() 
    ground_truth = ground_truth[ground_truth.nonzero()].flatten()
    return sqrt(mean_squared_error(prediction, ground_truth))

In [13]:
print('User-based CF RMSE: ' + str(rmse(user_prediction, test_data_matrix)))
print('Item-based CF RMSE: ' + str(rmse(item_prediction, test_data_matrix)))

User-based CF RMSE: 3.1187113781808784
Item-based CF RMSE: 3.446094080305067


Memory-based algorithms are easy to implement and produce reasonable prediction quality. 
The drawback of memory-based CF is that it doesn't scale to real-world scenarios and doesn't address the well-known cold-start problem, that is when new user or new item enters the system. Model-based CF methods are scalable and can deal with higher sparsity level than memory-based models, but also suffer when new users or items that don't have any ratings enter the system. I would like to thank Ethan Rosenthal for his [post](http://blog.ethanrosenthal.com/2015/11/02/intro-to-collaborative-filtering/) about Memory-Based Collaborative Filtering. 

# Model-based Collaborative Filtering

Model-based Collaborative Filtering is based on **matrix factorization (MF)** which has received greater exposure, mainly as an unsupervised learning method for latent variable decomposition and dimensionality reduction. Matrix factorization is widely used for recommender systems where it can deal better with scalability and sparsity than Memory-based CF. The goal of MF is to learn the latent preferences of users and the latent attributes of items from known ratings (learn features that describe the characteristics of ratings) to then predict the unknown ratings through the dot product of the latent features of users and items. 
When you have a very sparse matrix, with a lot of dimensions, by doing matrix factorization you can restructure the  user-item matrix into low-rank structure, and you can represent the matrix by the multiplication of two low-rank matrices, where the rows contain the latent vector. You fit this matrix to approximate your original matrix, as closely as possible, by multiplying the low-rank matrices together, which fills in the entries missing in the original matrix.

Let's calculate the sparsity level of MovieLens dataset:

In [14]:
sparsity=round(1.0-len(df)/float(n_users*n_items),3)
print('The sparsity level of MovieLens100K is ' +  str(sparsity*100) + '%')

The sparsity level of MovieLens100K is 93.7%


To give an example of the learned latent preferences of the users and items: let's say for the MovieLens dataset you have the following information: _(user id, age, location, gender, movie id, director, actor, language, year, rating)_. By applying matrix factorization the model learns that important user features are _age group (under 10, 10-18, 18-30, 30-90)_, _location_ and _gender_, and for movie features it learns that _decade_, _director_ and _actor_ are most important. Now if you look into the information you have stored, there is no such feature as the _decade_, but the model can learn on its own. The important aspect is that the CF model only uses data (user_id, movie_id, rating) to learn the latent features. If there is little data available model-based CF model will predict poorly, since it will be more difficult to learn the latent features. 

Models that use both ratings and content features are called **Hybrid Recommender Systems** where both Collaborative Filtering and Content-based Models are combined. Hybrid recommender systems usually show higher accuracy than Collaborative Filtering or Content-based Models on their own: they are capable to address the cold-start problem better since if you don't have any ratings for a user or an item you could use the metadata from the user or item to make a prediction. 

### SVD
A well-known matrix factorization method is **Singular value decomposition (SVD)**. Collaborative Filtering can be formulated by approximating a matrix `X` by using singular value decomposition. The winning team at the Netflix Prize competition used SVD matrix factorization models to produce product recommendations, for more information I recommend to read articles: [Netflix Recommendations: Beyond the 5 stars](http://techblog.netflix.com/2012/04/netflix-recommendations-beyond-5-stars.html) and [Netflix Prize and SVD](http://buzzard.ups.edu/courses/2014spring/420projects/math420-UPS-spring-2014-gower-netflix-SVD.pdf).
The general equation can be expressed as follows:
<img src="https://latex.codecogs.com/gif.latex?X=USV^T" title="X=USV^T" />


Given `m x n` matrix `X`:
* *`U`* is an *`(m x r)`* orthogonal matrix
* *`S`* is an *`(r x r)`* diagonal matrix with non-negative real numbers on the diagonal
* *V^T* is an *`(r x n)`* orthogonal matrix

Elements on the diagnoal in `S` are known as *singular values of `X`*. 


Matrix *`X`* can be factorized to *`U`*, *`S`* and *`V`*. The *`U`* matrix represents the feature vectors corresponding to the users in the hidden feature space and the *`V`* matrix represents the feature vectors corresponding to the items in the hidden feature space.
<img class="aligncenter size-thumbnail img-responsive" style="max-width:100%; width: 50%; max-width: none" src="http://s33.postimg.org/kwgsb5g1b/BLOG_CCA_5.png"/>

Now you can make a prediction by taking dot product of *`U`*, *`S`* and *`V^T`*.

<img class="aligncenter size-thumbnail img-responsive" style="max-width:100%; width: 50%; max-width: none" src="http://s33.postimg.org/ch9lcm6pb/BLOG_CCA_4.png"/>

In [15]:
import scipy.sparse as sp
from scipy.sparse.linalg import svds

#get SVD components from train matrix. Choose k.
u, s, vt = svds(train_data_matrix, k = 20)
s_diag_matrix=np.diag(s)
X_pred = np.dot(np.dot(u, s_diag_matrix), vt)
print('User-based CF MSE: ' + str(rmse(X_pred, test_data_matrix)))

User-based CF MSE: 2.716088818169583


Carelessly addressing only the relatively few known entries is highly prone to overfitting. SVD can be very slow and computationally expensive. More recent work minimizes the squared error by applying alternating least square or stochastic gradient descent and uses regularization terms to prevent overfitting. Alternating least square and stochastic gradient descent methods for CF will be covered in the next tutorials.


Review:

* We have covered how to implement simple **Collaborative Filtering** methods, both memory-based CF and model-based CF.
* **Memory-based models** are based on similarity between items or users, where we use cosine-similarity.
* **Model-based CF** is based on matrix factorization where we use SVD to factorize the matrix.
* Building recommender systems that perform well in cold-start scenarios (where little data is available on new users and items) remains a challenge. The standard collaborative filtering method performs poorly is such settings. 

## Looking for more?

If you want to tackle your own recommendation system analysis, check out these data sets. Note: The files are quite large in most cases, not all the links may stay up to host the data, but the majority of them still work. Or just Google for your own data set!

**Movies Recommendation:**

MovieLens - Movie Recommendation Data Sets http://www.grouplens.org/node/73

Yahoo! - Movie, Music, and Images Ratings Data Sets http://webscope.sandbox.yahoo.com/catalog.php?datatype=r

Jester - Movie Ratings Data Sets (Collaborative Filtering Dataset) http://www.ieor.berkeley.edu/~goldberg/jester-data/

Cornell University - Movie-review data for use in sentiment-analysis experiments http://www.cs.cornell.edu/people/pabo/movie-review-data/

**Music Recommendation:**

Last.fm - Music Recommendation Data Sets http://www.dtic.upf.edu/~ocelma/MusicRecommendationDataset/index.html

Yahoo! - Movie, Music, and Images Ratings Data Sets http://webscope.sandbox.yahoo.com/catalog.php?datatype=r

Audioscrobbler - Music Recommendation Data Sets http://www-etud.iro.umontreal.ca/~bergstrj/audioscrobbler_data.html

Amazon - Audio CD recommendations http://131.193.40.52/data/

**Books Recommendation:**

Institut für Informatik, Universität Freiburg - Book Ratings Data Sets http://www.informatik.uni-freiburg.de/~cziegler/BX/
Food Recommendation:

Chicago Entree - Food Ratings Data Sets http://archive.ics.uci.edu/ml/datasets/Entree+Chicago+Recommendation+Data
Merchandise Recommendation:

**Healthcare Recommendation:**

Nursing Home - Provider Ratings Data Set http://data.medicare.gov/dataset/Nursing-Home-Compare-Provider-Ratings/mufm-vy8d

Hospital Ratings - Survey of Patients Hospital Experiences http://data.medicare.gov/dataset/Survey-of-Patients-Hospital-Experiences-HCAHPS-/rj76-22dk

**Dating Recommendation:**

www.libimseti.cz - Dating website recommendation (collaborative filtering) http://www.occamslab.com/petricek/data/
Scholarly Paper Recommendation:

National University of Singapore - Scholarly Paper Recommendation http://www.comp.nus.edu.sg/~sugiyama/SchPaperRecData.html

# Great Job!